In [2]:


import SimpleITK as sitk
import numpy as np
import cv2
import os
import random
import shutil
from ultralytics import YOLO

import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import random
import pandas as pd



In [2]:
train_results = pd.read_csv('../radiological_gradings.csv')
train_results = train_results.loc[:, ['Patient','IVD label','Pfirrman grade']]
train_results

,Patient,IVD label,Pfirrman grade
0,1,1,3
1,1,2,3
2,1,3,3
3,1,4,4
4,1,5,4
...,...,...,...
1515,257,2,4
1516,257,3,4
1517,257,4,4
1518,257,5,2


In [3]:
transform_default = transforms.Compose([transforms.ToTensor()])

In [4]:
def prepare_transforms():
    
    train_set = ImageFolder(root='PFIRRMANN_dataset/train/', transform=transform_default)
    validate_set = ImageFolder(root='PFIRRMANN_dataset/val/', transform=transform_default)
        
    data_loader = DataLoader(train_set, batch_size=32, num_workers=6, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_test = DataLoader(validate_set, batch_size=128, num_workers=2, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)

    return data_loader, data_loader_test

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=5):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 128, kernel_size=8, padding=1)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(128, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2,2)

        # flatten: 32 * 15 * 15
        self.linear1 = nn.Linear(32*15*15, 100)
        self.linear2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = torch.flatten(x,1)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x


class CNN2(nn.Module):
    def __init__(self, in_channels=3, num_classes=5):
        super(CNN2, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, 128, kernel_size=8, padding=1)
        self.pool1 = nn.MaxPool2d(2,2)
        
        self.conv2 = nn.Conv2d(128, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2,2)
        
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(2,2)
        
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(32*7*7, 100)
        self.linear2 = nn.Linear(100, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = F.relu(self.conv4(x))
        x = self.pool4(x)
        x = self.flatten(x)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [6]:


model = CNN()



In [7]:
model

CNN(
  (conv1): Conv2d(3, 128, kernel_size=(8, 8), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=7200, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=5, bias=True)
)

In [6]:


def test(model,data_loader_test, criterion,device):
    test_loss = 0
    correct = 0
    total = 0
    for i, (batch_X, batch_Y) in enumerate(data_loader_test):
        X = batch_X.to(device, non_blocking=True)
        Y = batch_Y.to(device, non_blocking=True)
        outputs = model(X)
        loss = criterion(outputs, Y)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == Y).sum().item()
        total += Y.size(0)
    avg_test_loss = test_loss / len(data_loader_test)
    test_accuracy = correct / total * 100
    print(f" Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")            
    return  avg_test_loss, test_accuracy



In [7]:
torch.cuda.is_available()

True

In [8]:
def model_train(lr, model, seed, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

    
    losses_tr = []
    accuracies_tr = []
    losses_val = []
    accuracies_val = []
    data_loader, data_loader_test = prepare_transforms()
            
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
        
    num_epochs = num_epochs
    
    prev_prev_prev_loss = float('inf')
    prev_prev_loss = float('inf')
    prev_loss = float('inf')
    curr_loss = float('inf')

    if isinstance(model, type):
        model_instance = model()
    else:
        model_instance = model
    model_instance.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_instance.parameters(), lr=lr)
    for epoch in range(num_epochs): 
        print('Entered the loop')
        model_instance.train()
        total_loss = 0
        correct = 0
        total = 0

        loop = tqdm(enumerate(data_loader), total=len(data_loader), ncols=100, colour='green')
        for i, (batch_X, batch_Y) in loop:
            X = batch_X.to(device, non_blocking=True)  
            Y = batch_Y.to(device, non_blocking=True)  
            optimizer.zero_grad()
            outputs = model_instance(X)
            loss = criterion(outputs, Y)
            loss.backward()
            optimizer.step()
                
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == Y).sum().item()
            total += Y.size(0)

            loop.set_description(f"Train Loss: {total_loss/(i+1):.4f}, Acc: {100*correct/total:.2f}%")
                
    
        avg_loss = total_loss / len(data_loader)
        train_accuracy = correct / total * 100
        train_losses.append(avg_loss)
        train_accuracies.append(train_accuracy)
        print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")
            
            
        model_instance.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            loop_val = tqdm(enumerate(data_loader_test), total=len(data_loader_test), ncols=100, colour='blue')
            for i, (batch_X, batch_Y) in loop_val:
                X = batch_X.to(device, non_blocking=True)
                Y = batch_Y.to(device, non_blocking=True)
                outputs = model_instance(X)
                loss = criterion(outputs, Y)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == Y).sum().item()
                total += Y.size(0)

                loop_val.set_description(f"Val Loss: {val_loss/(i+1):.4f}, Acc: {100*correct/total:.2f}%")
        avg_val_loss = val_loss / (140)
        val_accuracy = correct / total * 100
        print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")            
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)
    
        
        if(epoch == num_epochs - 1 ):
            losses_tr.append(train_losses)
            accuracies_tr.append(train_accuracies) 
            losses_val.append(val_losses)  
            accuracies_val.append(val_accuracies) 
            avg_test_loss, test_accuracy = test(model_instance, data_loader_test, criterion, device)

    return model_instance,lr,val_losses, val_accuracies, train_losses, train_accuracies, avg_test_loss, test_accuracy

In [19]:


lrate = [0.001, 0.0001]
models= [CNN, CNN2]
results = []
for lr in lrate:
    for model_class in models:  # np. CNN
        model, lr_ret, val_losses, val_accuracies, train_losses, train_accuracies, avg_test_loss, test_accuracy = model_train(lr, model_class, 42, 30)
        
        # poprawna nazwa pliku
        model_filename = f"model_{model_class.__name__}_lr{lr_ret}.pt"
        torch.save(model.state_dict(), model_filename)
        
        results.append({
            "model_name": model_class.__name__,
            "learning_rate": lr_ret,
            "val_losses": val_losses,
            "val_accuracies": val_accuracies,
            "train_losses": train_losses,
            "train_accuracies": train_accuracies,
            "avg_test_loss": avg_test_loss,
            "test_accuracy": test_accuracy,
            "model_file": model_filename
        })


# zapis wszystkich wyników do CSV (jeśli nie potrzebujesz wszystkich list, np. tylko średnie)
df = pd.DataFrame(results)
df.to_csv("experiment_results.csv", index=False)


Entered the loop


Train Loss: 1.2708, Acc: 43.80%: 100%|██████████████████████████████| 86/86 [00:06<00:00, 13.05it/s]


Epoch 1, Average Loss: 1.2708


Val Loss: 1.1440, Acc: 48.91%: 100%|██████████████████████████████████| 6/6 [00:05<00:00,  1.18it/s]


Epoch 1, Validation Loss: 0.0490, Accuracy: 48.91%
Entered the loop


...

Epoch 29, Validation Loss: 0.1020, Accuracy: 51.09%
Entered the loop


Train Loss: 0.1812, Acc: 93.23%: 100%|██████████████████████████████| 86/86 [00:07<00:00, 11.98it/s]


Epoch 30, Average Loss: 0.1812


Val Loss: 2.2873, Acc: 50.51%: 100%|██████████████████████████████████| 6/6 [00:00<00:00,  6.28it/s]


Epoch 30, Validation Loss: 0.0980, Accuracy: 50.51%
 Test Loss: 2.3021, Test Accuracy: 50.51%


In [9]:
import torch
import torch.nn as nn

resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)

num_classes = 5
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)

dummy_input = torch.randn(1, 3, 128, 128)
output = resnet50(dummy_input)
print(output.shape)  


Using cache found in C:\Users\danci/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
C:\Users\danci/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
C:\Users\danci/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


torch.Size([1, 5])


In [10]:


lrate = [0.0001]
models= [resnet50]
results = []
for lr in lrate:
    for model_class in models:  
        model, lr_ret, val_losses, val_accuracies, train_losses, train_accuracies, avg_test_loss, test_accuracy = model_train(lr, model_class, 42, 10)
        
        
        model_filename = f"model_{model_class.__name__}_lr{lr_ret}.pt"
        torch.save(model.state_dict(), model_filename)
        
        results.append({
            "model_name": model_class.__name__,
            "learning_rate": lr_ret,
            "val_losses": val_losses,
            "val_accuracies": val_accuracies,
            "train_losses": train_losses,
            "train_accuracies": train_accuracies,
            "avg_test_loss": avg_test_loss,
            "test_accuracy": test_accuracy,
            "model_file": model_filename
        })


# zapis wszystkich wyników do CSV (jeśli nie potrzebujesz wszystkich list, np. tylko średnie)
df = pd.DataFrame(results)
df.to_csv("experiment_results_resnet.csv", index=False)


Entered the loop


Train Loss: 1.2474, Acc: 46.32%: 100%|██████████████████████████████| 86/86 [00:21<00:00,  4.00it/s]


Epoch 1, Average Loss: 1.2474


Val Loss: 1.0692, Acc: 59.85%: 100%|██████████████████████████████████| 6/6 [00:05<00:00,  1.07it/s]


Epoch 1, Validation Loss: 0.0458, Accuracy: 59.85%
Entered the loop


...

Train Loss: 0.0872, Acc: 96.96%: 100%|██████████████████████████████| 86/86 [00:22<00:00,  3.80it/s]


Epoch 10, Average Loss: 0.0872


Val Loss: 1.9552, Acc: 54.31%: 100%|██████████████████████████████████| 6/6 [00:01<00:00,  4.30it/s]


Epoch 10, Validation Loss: 0.0838, Accuracy: 54.31%


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 6.42 GiB is allocated by PyTorch, and 195.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [11]:














        model_filename = f"model_{model_class.__name__}_lr{lr_ret}.pt"
        torch.save(model.state_dict(), model_filename)
        
        results.append({
            "model_name": model_class.__name__,
            "learning_rate": lr_ret,
            "val_losses": val_losses,
            "val_accuracies": val_accuracies,
            "train_losses": train_losses,
            "train_accuracies": train_accuracies,
            "avg_test_loss": avg_test_loss,
            "test_accuracy": test_accuracy,
            "model_file": model_filename
        })


























































AttributeError: 'ResNet' object has no attribute '__name__'